In [1]:
# load the raw dataset
import numpy as np
import pandas as pd
from pandas import DataFrame

rawdf = pd.read_csv("unc.edu_PANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep="\t", index_col=0)
rawdf.head()

,TCGA-OR-A5J1-01A-11R-A29S-07,TCGA-OR-A5J2-01A-11R-A29S-07,TCGA-OR-A5J3-01A-11R-A29S-07,TCGA-OR-A5J5-01A-11R-A29S-07,TCGA-OR-A5J6-01A-31R-A29S-07,TCGA-OR-A5J7-01A-11R-A29S-07,TCGA-OR-A5J8-01A-11R-A29S-07,TCGA-OR-A5J9-01A-11R-A29S-07,TCGA-OR-A5JA-01A-11R-A29S-07,TCGA-OR-A5JB-01A-11R-A29S-07,...,TCGA-VQ-AA6A-01A-11R-A414-31,TCGA-VQ-AA6B-01A-11R-A414-31,TCGA-VQ-AA6D-01A-11R-A414-31,TCGA-VQ-AA6F-01A-31R-A414-31,TCGA-VQ-AA6G-01A-11R-A414-31,TCGA-VQ-AA6I-01A-11R-A414-31,TCGA-VQ-AA6J-01A-11R-A414-31,TCGA-VQ-AA6K-01A-11R-A414-31,TCGA-ZA-A8F6-01A-23R-A36D-31,TCGA-ZQ-A9CR-01A-11R-A39E-31
0.0gene_id,,,,,,,,,,,,,,,,,,,,,
?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.3189,0.0000,0.0000,0.0000,0.0000,0.2466,0.4134,0.0000,0.5668
?|100133144,3.2661,2.6815,1.7301,0.0000,0.0000,1.1673,1.4422,0.0000,4.4556,7.1293,...,26.1744,28.1937,32.4153,18.6091,33.7625,35.0189,27.6770,28.6855,29.3939,19.8016
?|100134869,3.9385,8.9948,6.5650,1.5492,4.4709,6.0529,2.2876,1.3599,5.0581,0.0000,...,14.3662,16.5156,21.5482,17.2753,20.2653,30.1605,19.2494,21.1378,15.4703,8.5379
?|10357,149.1350,81.0777,86.4879,53.9117,66.9063,103.5060,94.9316,78.1955,69.2389,155.7090,...,11.5541,10.0447,15.9546,21.9117,8.4115,21.6463,35.6665,11.5371,22.0386,29.3701
?|10431,2034.1000,1304.9300,1054.6600,2350.8900,1257.9900,1866.4300,995.0270,1762.1200,1213.5300,2005.5700,...,357.7860,780.6260,903.2380,1033.0300,733.7430,519.9930,709.6430,702.4730,561.9840,1874.0200


In [ ]:
# transpose raw dataset so row is patient samples and column is list of genes
processeddf = rawdf.transpose()
# load class label
dflabels = pd.read_csv('project_class_labels_original_10471.csv',index_col='Unnamed: 0')

In [ ]:
# check to see if there is any feature (i.e. column) has all zero values so we will delete them
removedAllZeroColdf = processeddf.loc[:, (processeddf != 0).any(axis=0)]
removedAllZeroColdf.shape

In [ ]:
# data scaling
# method 1 : standardization
from sklearn.preprocessing import StandardScaler

stdscaler = StandardScaler()
stdscalerfit = stdscaler.fit_transform(removedAllZeroColdf)

stddf = DataFrame(stdscalerfit,index = removedAllZeroColdf.index, columns = removedAllZeroColdf.columns)
stddf.head()

In [ ]:
# split 80% training set; 20% testing set
from sklearn.model_selection import train_test_split

trainData, testData, trainLabel, testLabel = train_test_split(stddf, dflabels, test_size=0.20)

In [ ]:
# svc model - Linear SVC (one vs rest)
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# measure training time
import time

lin_svm_clf = svm.LinearSVC()
start = time.time()
lin_svm_clf.fit(trainData, trainLabel.values.ravel())
end = time.time()
print("linear SVC training time: ", end - start)

pred = lin_svm_clf.predict(testData)
accuracy = accuracy_score(testLabel, pred)
print("Accuracy Score (linear kernel):", accuracy)

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(testLabel, pred)
print(confusion_matrix)
# classficiation report
from sklearn.metrics import classification_report
print(classification_report(testLabel, pred))

